With inspiration from : https://towardsdatascience.com/my-deep-learning-model-says-sorry-i-dont-know-the-answer-that-s-absolutely-ok-50ffa562cb0b

In [10]:
import sys
import os
import pandas as pd
import numpy as np

module_path = os.path.abspath(os.path.join('..'))

sys.path.insert(1, module_path + '/src')
import audio_time_series_classification as preproject
import utility

sys.path.insert(1, module_path + '/src/models/')
import helper

from sklearn.decomposition import PCA
from sktime.utils.data_io import load_from_tsfile_to_dataframe
from sklearn.feature_selection import SelectKBest, chi2
from sktime.utils.data_processing import from_nested_to_2d_array
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sktime.utils.data_io import load_from_tsfile_to_dataframe
import matplotlib.pyplot as plt
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import MinMaxScaler

figure_path = module_path + '/figures/'

from keras.utils import np_utils
import keras
from keras.models import Model
from keras.models import Sequential
from keras.layers import Conv1D, ZeroPadding1D, MaxPooling1D, BatchNormalization, Activation, Dropout, Flatten, Dense


In [5]:
X_train, y_train_o = load_from_tsfile_to_dataframe(module_path + f'/features/extracted_ts_files/UiT_allLabels_15s_TRAIN.ts')
X_test, y_test_o = load_from_tsfile_to_dataframe(module_path + f'/features/extracted_ts_files/UiT_allLabels_15s_TEST.ts')
X_val, y_val_o = load_from_tsfile_to_dataframe(module_path + f'/features/extracted_ts_files/UiT_allLabels_15s_VAL.ts')

X_train, X_test, X_val = from_nested_to_2d_array(X_train), from_nested_to_2d_array(X_test), from_nested_to_2d_array(X_val)

cols = np.arange(len(X_test.columns))
X_train.columns, X_test.columns, X_val.columns = cols, cols, cols

y_train_o, y_test_o, y_val_o = pd.Series(y_train_o), pd.Series(y_test_o), pd.Series(y_val_o)

scaler = MinMaxScaler() 
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train)) 
X_val = pd.DataFrame(scaler.transform(X_val)) 
X_test = pd.DataFrame(scaler.transform(X_test)) 

In [7]:
y_train_new = y_train_o.copy(deep = True)

y_train_new[y_train_new == 'exp_wheeze'] = 'wheeze'
y_train_new[y_train_new == 'insp_wheeze'] = 'wheeze'
y_train_new[y_train_new == 'exp_crackle'] = 'crackle'
y_train_new[y_train_new == 'insp_crackle'] = 'crackle'

y_test_new = y_test_o.copy(deep = True)

y_test_new[y_test_new == 'exp_wheeze'] = 'wheeze'
y_test_new[y_test_new == 'insp_wheeze'] = 'wheeze'
y_test_new[y_test_new == 'exp_crackle'] = 'crackle'
y_test_new[y_test_new == 'insp_crackle'] = 'crackle'

y_val_new = y_val_o.copy(deep = True)

y_val_new[y_val_new == 'exp_wheeze'] = 'wheeze'
y_val_new[y_val_new == 'insp_wheeze'] = 'wheeze'
y_val_new[y_val_new == 'exp_crackle'] = 'crackle'
y_val_new[y_val_new == 'insp_crackle'] = 'crackle'

indices_1 = np.where(y_train_new == 'normal')[0]
indices_2 = np.where(y_train_new == 'wheeze')[0]
indices_3 = np.where(y_train_new == 'crackle')[0]
#indices_train = np.concatenate((indices_2, indices_3))
indices_train = np.concatenate((indices_1[:250], indices_2, indices_3))

indices_1 = np.where(y_test_new == 'normal')[0]
indices_2 = np.where(y_test_new == 'wheeze')[0]
indices_3 = np.where(y_test_new == 'crackle')[0]
#indices_test = np.concatenate((indices_2, indices_3))
indices_test = np.concatenate((indices_1, indices_2, indices_3))

indices_1 = np.where(y_val_new == 'normal')[0]
indices_2 = np.where(y_val_new == 'wheeze')[0]
indices_3 = np.where(y_val_new == 'crackle')[0]
#indices_val = np.concatenate((indices_2, indices_3))
indices_val = np.concatenate((indices_1, indices_2, indices_3))

y_train = y_train_new[indices_train]
y_test = y_test_new[indices_test]
y_val = y_val_new[indices_val]

le = preprocessing.LabelEncoder()

le.fit(y_train_new[indices_train])
num_classes = len(le.classes_)
y_train = le.transform(y_train_new[indices_train])
y_test = le.transform(y_test_new[indices_test])
y_val = le.transform(y_val_new[indices_val])

In [11]:
model = Sequential()
model.add(Dense(200, activation='relu'))
model.add(BatchNormalization()) 
model.add(Dense(200, activation='relu'))
model.add(BatchNormalization()) 
model.add(Dense(200, activation='relu'))
model.add(BatchNormalization()) 
model.add(Dense(num_classes, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [16]:
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.regularizers import l2

# function for creating a fully connected neural network of any architecture
# the number of neurons in each layer is defined by layers_shape
# the droupout_proba indicates if dropout layers should be added
def architecture(layers_shape, input_dim, output_dim, dropout_proba, reg, act='relu', verbose=False):
    inputs = Input(shape=(input_dim,))
    hidden = Dense(layers_shape[0], activation=act,
                   kernel_regularizer=l2(reg))(inputs)
    for i in range(len(layers_shape)-1):
        if dropout_proba > 0:
            hidden = Dropout(dropout_proba)(hidden, training=True)
        hidden = Dense(layers_shape[i+1], activation=act, kernel_regularizer=l2(reg))(hidden)
    if dropout_proba > 0:
        hidden = Dropout(dropout_proba)(hidden, training=True)
    outputs = Dense(output_dim, kernel_regularizer=l2(reg))(hidden) 
    model = Model(inputs, outputs)
    if verbose:
        model.summary()
    return model
  
model_without_dropout = architecture(layers_shape=[100, 50, 10], input_dim= 218, output_dim=2, 
                                     dropout_proba=0, reg=0, act='relu', verbose=1)

model_with_dropout = architecture(layers_shape=[100, 50, 10], input_dim= 218, output_dim=2, 
                                  dropout_proba=0.05, reg=0.00475, act='relu', verbose=1)

here
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 218)]             0         
_________________________________________________________________
dense_32 (Dense)             (None, 100)               21900     
_________________________________________________________________
dense_33 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_34 (Dense)             (None, 10)                510       
_________________________________________________________________
dense_35 (Dense)             (None, 2)                 22        
Total params: 27,482
Trainable params: 27,482
Non-trainable params: 0
_________________________________________________________________
here
Model: "model_7"
_________________________________________________________________
Layer (type)                

In [6]:
import random
from sklearn import metrics

from fylearn.nfpc import FuzzyPatternClassifier
from fylearn.fpt import FuzzyPatternTreeTopDownClassifier

random.seed(0)

clf = FuzzyPatternTreeTopDownClassifier()

auc = []
ignorance = []
conflict = []

indices_0 = np.where(y_train.astype(int) == 0)[0]
indices_1 = np.where(y_train.astype(int) == 1)[0]
indices = np.concatenate((indices_0[0:1929], indices_1))


for i in range(100, len(indices), 100):

    p = indices.copy()
    random.shuffle(p)
    indices_2 = p[0:i]
    

    clf.fit(X_train.iloc[indices_2], y_train.iloc[indices_2])

    pred, t = clf.predict(X_test)
    
    auc_score = metrics.roc_auc_score(y_test, pred)
    
    ig = np.where((((t< 0.45)[: , 0]) & ((t< 0.45)[: , 1])) == True)[0]
    ignorant_indices = len(ig)
    print(ig)

    conf = np.where((((t> 0.55)[: , 0]) & ((t> 0.55)[: , 1])) == True)[0]
    conflict_indices = len(conf)
    print(conflict_indices)
    
    conflict.append(conflict_indices)
    ignorance.append(ignorant_indices)
    auc.append(auc_score)
    #break


[   5   19   30   65   90   93  113  165  169  183  192  196  221  261
  281  296  310  317  386  396  398  412  427  429  438  453  494  508
  565  586  609  628  653  660  670  676  703  708  712  724  735  765
  778  816  819  831  849  857  867  905  926  942  943  975  977  979
  992  997 1030 1032 1041 1047 1056 1082 1096 1111 1117 1155 1167 1178
 1180 1214 1215 1231 1239 1253 1263 1271 1301 1303 1338 1341 1348]
99
[   8   12   32   36   41   42   50   53   60   75   90  108  122  123
  140  161  167  177  179  192  195  227  237  242  265  270  281  294
  303  305  312  315  330  332  336  343  344  356  361  372  376  377
  393  409  413  414  421  423  445  456  462  463  480  481  485  498
  503  508  528  530  532  533  554  562  568  580  615  621  652  663
  672  675  686  690  695  703  712  714  724  728  739  740  793  804
  805  813  819  831  845  847  868  884  890  899  901  921  943  944
  965  970  978  982  986  992  993  995 1000 1007 1009 1021 1029 1034
 1038 1